# European Football ranking over the time period(2008-2016)

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

>In this project, we will find out european soccer ranking. During analyzing the data, we will try to find out:  
- Which league scores the most among the period? 
- Is there a home advantage? 
- Which team wins the most? 
- The best team by each league? 


<a id='wrangling'></a>
## Data Wrangling

### 1) Data Preparation 
> I imported all required libraries for reading data, analysing and visualizing data below. I would like to use plot.ly for personal practice, so I will add plot.ly charts for the data visualization.

In [37]:
# Import libraries 
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.subplots import make_subplots

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

path = "../input/"  #Insert path here
database = path + 'database.sqlite'

In [2]:
# load data
path = '/Users/sojung/projects/database.sqlite'
with sqlite3.connect(path) as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    players = pd.read_sql_query('SELECT * from Player', con)
    players_attributes = pd.read_sql_query('SELECT * from Player_Attributes', con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    teams_attributes = pd.read_sql_query("SELECT * from Team_Attributes", con)

### 2) General Properties



#### Country Table 

In [3]:
countries # 11 countries 

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


#### League Table 

In [4]:
leagues # league id and country_id are same and also there are same number of leauges 

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


#### Match Table

In [5]:
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [6]:
#Check all the names of columns in list
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB


In [7]:
matches.shape # 115 columns and 25979 data 

(25979, 115)

In [8]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB


Since there are 115 columns in this tables, we will take only few columns later. 

#### Players Table 

In [9]:
players.head() 

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


We have 2 IDs for each players; player_api_id and player_fifa_api_id so we will decide which ID will be used and drop the other ID later. 

#### Players Attribute Table 

In [10]:
players_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [11]:
players_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
id                     183978 non-null int64
player_fifa_api_id     183978 non-null int64
player_api_id          183978 non-null int64
date                   183978 non-null object
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    180748 non-null object
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                181265 non-null float64
dribbling              183142 non-null float64
curve                  181265 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
accele

Not relevant data for our analysis. Will not use. 

#### Teams Table

In [12]:
teams.head() # 2 IDs for team - team_api_id & team_fifa_api_id

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [13]:
teams.shape # 5 columns and 299 data 

(299, 5)

In [14]:
teams.info() # 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
id                  299 non-null int64
team_api_id         299 non-null int64
team_fifa_api_id    288 non-null float64
team_long_name      299 non-null object
team_short_name     299 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


We can see there is missing data in team_fifa_api_id. Thus, We will use team_api_id for the analysis. Check below. 

In [15]:
teams[teams['team_fifa_api_id'].isnull()] 

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
8,9,7947,NaN,FCV Dender EH,DEN
14,15,4049,NaN,Tubize,TUB
170,26561,6601,NaN,FC Volendam,VOL
204,34816,177361,NaN,Termalica Bruk-Bet Nieciecza,TBN
208,35286,7992,NaN,Trofense,TRO
213,35291,10213,NaN,Amadora,AMA
223,36248,9765,NaN,Portimonense,POR
225,36723,4064,NaN,Feirense,FEI
232,38789,6367,NaN,Uniao da Madeira,MAD
233,38791,188163,NaN,Tondela,TON


#### Teams Attributes Table

In [16]:
teams_attributes.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [17]:
teams_attributes.shape

(1458, 25)

### 3) Data Cleaning 

In [18]:
#Merge country and leauge data
countries_leagues = countries.merge(leagues,left_on="id",right_on="id",how="inner")
countries_leagues = countries_leagues.drop("id",axis = 1)
countries_leagues = countries_leagues.rename(columns={'name_x':"country", 'name_y':"league"})

In [19]:
countries_leagues 

,country,country_id,league
0,Belgium,1,Belgium Jupiler League
1,England,1729,England Premier League
2,France,4769,France Ligue 1
3,Germany,7809,Germany 1. Bundesliga
4,Italy,10257,Italy Serie A
5,Netherlands,13274,Netherlands Eredivisie
6,Poland,15722,Poland Ekstraklasa
7,Portugal,17642,Portugal Liga ZON Sagres
8,Scotland,19694,Scotland Premier League
9,Spain,21518,Spain LIGA BBVA


In [20]:
#subsetting data with necessary columns
matches_new = matches[['id', 'country_id', 'league_id', 'season', 'stage', 'date',
                   'match_api_id', 'home_team_api_id', 'away_team_api_id',
                    'home_team_goal', 'away_team_goal']]

matches_new = matches_new.drop("id",axis=1)

In [21]:
#merge league data with match data
df = matches_new.merge(countries_leagues,left_on="country_id",right_on="country_id",how="outer")
#chech null values
df.isnull().sum()

country_id          0
league_id           0
season              0
stage               0
date                0
match_api_id        0
home_team_api_id    0
away_team_api_id    0
home_team_goal      0
away_team_goal      0
country             0
league              0
dtype: int64

In [22]:
#Unique values in dataframe
df.nunique()

country_id             11
league_id              11
season                  8
stage                  38
date                 1694
match_api_id        25979
home_team_api_id      299
away_team_api_id      299
home_team_goal         11
away_team_goal         10
country                11
league                 11
dtype: int64

In [23]:
df.head() # final dataset to analyse 

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,country,league
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,Belgium,Belgium Jupiler League
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,Belgium,Belgium Jupiler League
2,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,Belgium,Belgium Jupiler League
3,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,Belgium,Belgium Jupiler League
4,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,Belgium,Belgium Jupiler League


<a id='eda'></a>

## Exploratory Data Analysis

### 1. Which league scores the most? 

In [26]:
# group together by league and get the total goals from home team and away team 
goals_df = df.groupby("league").agg({"home_team_goal":"sum","away_team_goal":"sum"})\
                               .sort_values('home_team_goal',ascending=True)
goals_df

,home_team_goal,away_team_goal
league,,
Switzerland Super League,2365,1801
Scotland Premier League,2607,2197
Poland Ekstraklasa,2678,1978
Belgium Jupiler League,2781,2060
Portugal Liga ZON Sagres,2890,2311
Germany 1. Bundesliga,3982,3121
France Ligue 1,4265,3162
Netherlands Eredivisie,4357,3185
Italy Serie A,4528,3367


In [27]:
#caculate the total goals by leagues 
goals_df['total_goals']= goals_df.iloc[:, 0:2].sum(axis=1)
goals_df

,home_team_goal,away_team_goal,total_goals
league,,,
Switzerland Super League,2365,1801,4166
Scotland Premier League,2607,2197,4804
Poland Ekstraklasa,2678,1978,4656
Belgium Jupiler League,2781,2060,4841
Portugal Liga ZON Sagres,2890,2311,5201
Germany 1. Bundesliga,3982,3121,7103
France Ligue 1,4265,3162,7427
Netherlands Eredivisie,4357,3185,7542
Italy Serie A,4528,3367,7895


In [28]:
# horizontal bar chart in descending order 
fig = go.Figure()

data=goals_df

fig.add_trace(go.Bar(x=goals_df['home_team_goal'], y = list(goals_df.index),
                    name='Home Team Goal',
                    orientation = 'h',
                    marker_color='rgb(26, 118, 255)'
                    )
             )

fig.add_trace(go.Bar(x=goals_df['away_team_goal'], y = list(goals_df.index),
                    name='Away Team Goal',
                    orientation = 'h',
                    marker_color='rgb(55, 83, 109)'
                    )
             )

fig.add_trace(go.Bar(x=goals_df['total_goals'], y = list(goals_df.index),
                    name='Total Goal',
                    orientation = 'h',
                    marker_color='rgb(70, 210, 100)'
                    )
             )

fig.update_layout(
    title='Home and Away Team Goals by League',
    xaxis=dict(
        title='Goals',
        titlefont_size=14,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Leagues',
        titlefont_size=16,
        tickfont_size=14,
    ),
    barmode='group',
    bargap=0.20, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
fig.show()

We find that Spain LIGA BBVA scores the most with total 8,412 goals followed by England Premier league with total 8,240 goals. 

### 2. Is there a home advantage? 

In [29]:
#Merge home team data
df_copied = df.copy() # for backup
df_copied = df_copied.merge(teams,left_on='home_team_api_id',right_on='team_api_id',how='left') 
df_copied = df_copied.drop(['id','team_api_id','team_fifa_api_id','team_short_name'],axis = 1) # drop columns we will not use
df_copied = df_copied.rename(columns={'team_long_name':'home_team_name'}) 

df_copied.columns 

Index(['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       'away_team_goal', 'country', 'league', 'home_team_name'],
      dtype='object')

In [30]:
#Another join to merge away team data  
df_copied = df_copied.merge(teams,left_on='away_team_api_id',right_on='team_api_id',how='left')
df_copied = df_copied.drop(['id','team_api_id','team_fifa_api_id','team_short_name'],axis = 1)
df_copied = df_copied.rename(columns={'team_long_name':"away_team_name"})

df_copied.columns

Index(['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       'away_team_goal', 'country', 'league', 'home_team_name',
       'away_team_name'],
      dtype='object')

In [31]:
#Check top 10 home teams and away teams by descending orders of the number of the goals 
home_goals = df_copied.groupby('home_team_name')['home_team_goal'].sum().reset_index()
away_goals = df_copied.groupby('away_team_name')['away_team_goal'].sum().reset_index()
home_goals = home_goals.sort_values('home_team_goal',ascending=True)
away_goals = away_goals.sort_values('away_team_goal',ascending=True)
home_goals_2 = home_goals[-10:] 
away_goals_2 = away_goals[-10:]

In [38]:
#Create 2 subplots 

fig = make_subplots(rows=1, cols=2, subplot_titles=("Top 10 Home Team","Top 10 Away Team"))

fig.add_trace(go.Bar(x=home_goals_2.home_team_goal, y=home_goals_2.home_team_name,
                     name='Home Team Goal',
                     orientation = 'h',
                     marker_color='indianred'
                     ),
              1, 1)

fig.add_trace(go.Bar(x=away_goals_2.away_team_goal, y=away_goals_2.away_team_name,
                     name='Away Team Goal',
                     orientation = 'h',
                     marker_color='lightsalmon'
                     ),
              1, 2)

fig.update_layout(
    xaxis=dict(
        title='Home Team Goals',
        titlefont_size=12,
        tickfont_size=10,
    ),
    xaxis2=dict(
        title='Away Team Goals',
        titlefont_size=12,
        tickfont_size=10,
    ),
    yaxis=dict(
        titlefont_size=12,
        tickfont_size=10,
    ),
    yaxis2=dict(
        titlefont_size=12,
        tickfont_size=10,
    ),
    legend=dict(x=0.90, y=1.2, font_size=10),
    margin=dict(l=10, r=10, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

fig.show()


Real Madrid CF performed the best when they were playing games at home, which scores 505 goals, while FC Barcelona played well outside of the home, which scores 354 goals.  

In [39]:
#to check the result of games - home team win, away team win, draw
def match_result(row):  
    if row['home_team_goal'] == row['away_team_goal']:
        return 'Draw'
    elif row['home_team_goal'] > row['away_team_goal']:
        return 'Home team win'
    elif row['home_team_goal'] < row['away_team_goal']:
        return 'Away team win'
    
#Winning team names
def win_team(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return row['home_team_name']
    elif row['away_team_goal'] > row['home_team_goal']:
        return row['away_team_name']
    elif row['home_team_goal'] == row['away_team_goal']:
        return 'Draw'
    
#Losing team names 
def lost_team(row):
    if row['home_team_goal'] < row['away_team_goal']:
        return row['home_team_name']
    elif row['away_team_goal'] < row['home_team_goal']:
        return row['away_team_name']
    elif row['home_team_goal'] == row['away_team_goal']:
        return 'Draw'

In [41]:
match_df = df_copied.copy() # for backup

#Create new columns for winning team names, result, losing team names
match_df['win'] = match_df.apply(lambda row: win_team(row),axis=1)
match_df['result'] = match_df.apply(lambda row: match_result(row), axis = 1)
match_df['lost'] = match_df.apply(lambda row: lost_team(row),axis=1)

match_df

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,country,league,home_team_name,away_team_name,win,result,lost
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC,Draw,Draw,Draw
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren,Draw,Draw,Draw
2,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht,RSC Anderlecht,Away team win,KSV Cercle Brugge
3,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons,KAA Gent,Home team win,RAEC Mons
4,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège,Standard de Liège,Away team win,FCV Dender EH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0,Switzerland,Switzerland Super League,FC St. Gallen,FC Thun,FC St. Gallen,Home team win,FC Thun
25975,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,2,Switzerland,Switzerland Super League,FC Vaduz,FC Luzern,FC Luzern,Away team win,FC Vaduz
25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,0,Switzerland,Switzerland Super League,Grasshopper Club Zürich,FC Sion,Grasshopper Club Zürich,Home team win,FC Sion
25977,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,0,Switzerland,Switzerland Super League,Lugano,FC Zürich,Draw,Draw,Draw


In [42]:
#Pie chart
labels = match_df['result'].value_counts().index.tolist()
values = match_df['result'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()

We see that 45.9 percentage of the results is home wins, 25.4% are draws and the away team wins only 28.7% of the time. We could conclude that it is likely that homeadvatage exists. 

### 3. Which team wins the most?

In [43]:
match_copied = match_df.copy() # for backup
match_copied

,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,country,league,home_team_name,away_team_name,win,result,lost
0,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,Belgium,Belgium Jupiler League,KRC Genk,Beerschot AC,Draw,Draw,Draw
1,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,Belgium,Belgium Jupiler League,SV Zulte-Waregem,Sporting Lokeren,Draw,Draw,Draw
2,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,Belgium,Belgium Jupiler League,KSV Cercle Brugge,RSC Anderlecht,RSC Anderlecht,Away team win,KSV Cercle Brugge
3,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,Belgium,Belgium Jupiler League,KAA Gent,RAEC Mons,KAA Gent,Home team win,RAEC Mons
4,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,Belgium,Belgium Jupiler League,FCV Dender EH,Standard de Liège,Standard de Liège,Away team win,FCV Dender EH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0,Switzerland,Switzerland Super League,FC St. Gallen,FC Thun,FC St. Gallen,Home team win,FC Thun
25975,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,2,Switzerland,Switzerland Super League,FC Vaduz,FC Luzern,FC Luzern,Away team win,FC Vaduz
25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,0,Switzerland,Switzerland Super League,Grasshopper Club Zürich,FC Sion,Grasshopper Club Zürich,Home team win,FC Sion
25977,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,0,Switzerland,Switzerland Super League,Lugano,FC Zürich,Draw,Draw,Draw


In [54]:
# Count how many times each team win and lose 
win = match_copied["win"].value_counts()[1:].reset_index()
lost = match_copied["lost"].value_counts()[1:].reset_index()

In [45]:
#Create team dataframe
h_t = match_copied.groupby(["home_team_name","league"]).agg({"home_team_goal":"sum"}).reset_index()
a_t = match_copied.groupby(["away_team_name","league"]).agg({"away_team_goal":"sum"}).reset_index()
h_t = h_t.rename(columns={'home_team_name':"team", 'home_team_goal':"goals"})
a_t = a_t.rename(columns={'away_team_name':"team", 'away_team_goal':"goals"})
team_df = pd.concat([h_t,a_t])
team_df = team_df.groupby(["team","league"])[["goals"]].sum().reset_index()
team_df = team_df.sort_values(by="goals",ascending=False)

In [55]:
#merge win,draw and lost data of team to matches played
##merge with win data 
team = pd.merge(team_df, win, left_on='team', right_on='index', how='left')
team = team.drop("index",axis =1)
##merge with lost data 
team = pd.merge(team, lost, left_on="team", right_on="index", how="left")
team = team.drop("index",axis =1)
##merge with draw data 
draw = match_copied[match_copied["result"] == "Draw"][["home_team_name","away_team_name"]]

#seperate with home team and away team draw 
home_draw = draw["home_team_name"].value_counts().reset_index()
away_draw = draw["away_team_name"].value_counts().reset_index()
home_draw  = home_draw.rename(columns={'index':"team", 'home_team_name':'draw'})
away_draw  = away_draw.rename(columns={'index':"team", 'away_team_name':"draw"})

#create draw dataframe 
draw_team = pd.concat([home_draw,away_draw])
draw_team  = draw_team .groupby("team")["draw"].sum().reset_index()

#merge with team dataframe we've created above with draw dataframe
team = pd.merge(team, draw_team, left_on="team", right_on="team", how ="left")

team

,team,league,goals,win,lost,draw
0,FC Barcelona,Spain LIGA BBVA,849,234,27,43
1,Real Madrid CF,Spain LIGA BBVA,843,228,40,36
2,Celtic,Scotland Premier League,695,218,36,50
3,FC Bayern Munich,Germany 1. Bundesliga,653,193,35,44
4,PSV,Netherlands Eredivisie,652,178,47,47
...,...,...,...,...,...,...
291,Amadora,Portugal Liga ZON Sagres,26,8,12,10
292,Trofense,Portugal Liga ZON Sagres,25,5,17,8
293,FC Dordrecht,Netherlands Eredivisie,24,4,22,8
294,Córdoba CF,Spain LIGA BBVA,22,3,24,11


In [59]:
team = team.sort_values(by="goals",ascending=False) # sorted values in descending orders 
team_copied = team.copy() # backup 

#change team names as index 
team_copied.index = team_copied.team 

In [49]:
#select only top 20 teams 
team_sorted = team_copied[["win","lost","draw"]][:20]
team_sorted

,win,lost,draw
team,,,
FC Barcelona,234,27,43
Real Madrid CF,228,40,36
Celtic,218,36,50
FC Bayern Munich,193,35,44
PSV,178,47,47
Ajax,181,32,59
FC Basel,180,42,64
Manchester City,175,68,61
Chelsea,176,58,70


In [50]:
#bar chart of team's performance
x=team_sorted.index.tolist()

fig = go.Figure(go.Bar(x=x, y=team_sorted.win, name='Win'))
fig.add_trace(go.Bar(x=x, y=team_sorted.lost, name='Lost'))
fig.add_trace(go.Bar(x=x, y=team_sorted.draw, name='Draw'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

We find that FC Barcelona wins the most with 234 goals followed by Real Madrid CF.

### 4. The best team by each league 

In [51]:
#Create dataframe with league and counts how many times they win 
win_team_by_league = pd.DataFrame(match_copied.groupby(["league","win"])["win"].count())
win_team_by_league = win_team_by_league.rename(columns={"win":"team"}).reset_index()
#change column name
win_team_by_league = win_team_by_league.rename(columns={"win":"team","team":"win"})
win_team_by_league = win_team_by_league.sort_values(by="win",ascending=False)
#getting rid of the value has "Draw"
win_team_by_league = win_team_by_league[win_team_by_league["team"] != "Draw"]
win_team_by_league = win_team_by_league.drop_duplicates(subset=["league"],keep="first")

win_team_by_league

,league,team,win
265,Spain LIGA BBVA,FC Barcelona,234
240,Scotland Premier League,Celtic,218
108,Germany 1. Bundesliga,FC Bayern Munich,193
43,England Premier League,Manchester United,192
143,Italy Serie A,Juventus,189
231,Portugal Liga ZON Sagres,SL Benfica,185
162,Netherlands Eredivisie,Ajax,181
295,Switzerland Super League,FC Basel,180
85,France Ligue 1,Paris Saint-Germain,175
195,Poland Ekstraklasa,Legia Warszawa,137


In [60]:
#horizontal bar chart
fig = go.Figure(go.Bar(
            x=win_team_by_league.win,
            y=win_team_by_league.league,
            text=win_team_by_league.team.tolist(),
            marker_color='rgb(70, 103, 102)',
            orientation='h'))

fig.update_traces(textposition='inside')
fig.update_layout(
    title='Top teams by Leagues',
    
    xaxis=dict(
    title='Goals',
    titlefont_size=14,
    tickfont_size=14,
    ),

    yaxis=dict(
        autorange='reversed',
        title='Leagues',
        titlefont_size=16,
        tickfont_size=14,
    ))

fig.show()

In Spain LIGA BBVA, FC Barcelona is the best team, followed by Celtic in Scotland Premier League. 


<a id='Conclusions'></a>
## Conclusions

We have all the answers for 4 questions at the beginning. 

- Which league scores the most among the period? Spain LIGA BBVA 
- Is there a home advantage? Yes. Almost double possibilities of winning when team plays at home. 
- Which team wins the most? FC Barcelona. It's no doubt that Spain LIGA BBVA is the best league among European football leagues. 
- The best team by each league?\
  - Spain LIGA BBVA - FC Barcelona 
  - Scotland Premier League - Celtic 
  - Bundesliga - FC Bayern Munich
  - English Premier League - Manchester United 
  - Italy Series A - Juventus
  - Portugal Liga ZON Sagres - SL Benfica
  - Netherlands Eredivisie - Ajax
  - Swizerland Super League - FC Basel
  - France Ligue 1 - Paris Saint-Germain
  - Poland Ekstraklasa - Legia Warszawa
  - Belgium Juplier League - RSC Anderlecht 